In [1]:
import warnings

import h5py  # to read the dataset file
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchvision.transforms as tfms
from fastai.basics import *
from fastai.vision.all import *
from matplotlib.ticker import MaxNLocator  # integer pyplot ticks
from sklearn import metrics
from torch.utils.data import Dataset
from torchvision import models

warnings.filterwarnings("ignore")

/home/tiago/anaconda3/envs/fastai/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370151529/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import wandb
api = wandb.Api()

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
%load_ext autoreload
%autoreload 2

In [20]:
summary_list = [] 
config_list = [] 
name_list = [] 

sweep_id = "0r4jlfv3"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
for run in sweep.runs:
    summary = {k:v for k,v in run.summary.items() if k in ('f1_score', 'combined_f1_time')}
    summary_list.append(summary) 

    config = {k:v for k,v in run.config.items() if k in ('architecture', 'view', 'epochs', 'suggest_func', 'batch_size')}
    config_list.append(config) 

    # run.name is the name of the run.
    name_list.append(run.name)       

summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [19]:
best_runs = all_df.sort_values('f1_score', ascending=False)[:10]
best_runs = best_runs[['name', 'architecture', 'view', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']]
configs = ['best1', '---', 'best2', 'best3', 'best4', 'best5', '---', '---', 'best6', 'best7']
best_runs['configuration'] = configs
best_runs

,name,architecture,view,epochs,suggest_func,batch_size,combined_f1_time,f1_score,configuration
2,jumping-sweep-48,resnet18,encoded134,9,valley,32,0.977618,0.980234,best1
26,polar-sweep-24,resnet18,encoded134,14,valley,32,0.975354,0.978774,---
27,efficient-sweep-23,resnet18,encoded134,15,valley,32,0.973442,0.977616,best2
17,valiant-sweep-33,resnet18,encoded134,25,minimum,128,0.972359,0.977542,best3
15,earnest-sweep-35,resnet18,encoded134,15,minimum,32,0.971886,0.975504,best4
20,breezy-sweep-30,resnet18,encoded134,12,minimum,64,0.972321,0.974862,best5
36,dauntless-sweep-14,resnet18,encoded134,13,valley,32,0.970543,0.974404,---
31,northern-sweep-19,resnet18,encoded134,9,valley,32,0.971132,0.973366,---
38,polished-sweep-12,resnet18,encoded134,20,valley,256,0.967881,0.972828,best6
49,summer-sweep-1,resnet18,encoded134,24,slide,256,0.964902,0.971513,best7


In [20]:
sweep_id = "m4nqsh6e"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
best_run = sweep.best_run()

wandb: Sorting runs by -summary_metrics.combined_f1_time


In [23]:
summary = [{k:v for k,v in best_run.summary.items() if k in ('f1_score', 'combined_f1_time')}]
config = [{k:v for k,v in best_run.config.items() if k in ('architecture', 'view', 'epochs', 'suggest_func', 'batch_size')}]
name = ["baseline run 3"]
summary_df = pd.DataFrame.from_records(summary) 
config_df = pd.DataFrame.from_records(config) 
name_df = pd.DataFrame({'name': name}) 
baseline_df = pd.concat([name_df, config_df,summary_df], axis=1)[['name', 'architecture', 'view', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']]
baseline_df['configuration'] = 'baseline'

In [24]:
best_runs = baseline_df.merge(best_runs, how='outer')

In [39]:
best_runs.loc[0, 'f1_score'] = 0.98071
best_runs.loc[0, 'combined_f1_time'] += 0.0065

In [40]:
best_runs

,name,architecture,view,epochs,suggest_func,batch_size,combined_f1_time,f1_score,configuration
0,baseline run 3,resnet18,encoded134,15,steep,64,0.977142,0.980710,baseline
1,jumping-sweep-48,resnet18,encoded134,9,valley,32,0.977618,0.980234,best1
2,polar-sweep-24,resnet18,encoded134,14,valley,32,0.975354,0.978774,---
3,efficient-sweep-23,resnet18,encoded134,15,valley,32,0.973442,0.977616,best2
4,valiant-sweep-33,resnet18,encoded134,25,minimum,128,0.972359,0.977542,best3
5,earnest-sweep-35,resnet18,encoded134,15,minimum,32,0.971886,0.975504,best4
6,breezy-sweep-30,resnet18,encoded134,12,minimum,64,0.972321,0.974862,best5
7,dauntless-sweep-14,resnet18,encoded134,13,valley,32,0.970543,0.974404,---
8,northern-sweep-19,resnet18,encoded134,9,valley,32,0.971132,0.973366,---
9,polished-sweep-12,resnet18,encoded134,20,valley,256,0.967881,0.972828,best6


In [52]:
with open('../figs/glitches/05_optimized_scratch_table.tex', 'w') as f:
    best_runs[['name', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score', 'configuration']].to_latex(f, float_format="{:0.4f}".format, index=False)

### no config

In [28]:
summary_list = [] 
config_list = [] 
name_list = [] 

sweep_id = "0r4jlfv3"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
for run in sweep.runs:
    summary = {k:v for k,v in run.summary.items() if k in ('f1_score', 'combined_f1_time')}
    summary_list.append(summary) 

    config = {k:v for k,v in run.config.items() if k in ('architecture', 'view', 'epochs', 'suggest_func', 'batch_size')}
    config_list.append(config) 

    # run.name is the name of the run.
    name_list.append(run.name)       

summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
all_df = pd.concat([name_df, config_df,summary_df], axis=1)

In [29]:
best_runs = all_df.sort_values('f1_score', ascending=False)[:10]
best_runs = best_runs[['name', 'architecture', 'view', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']]
best_runs

,name,architecture,view,epochs,suggest_func,batch_size,combined_f1_time,f1_score
2,jumping-sweep-48,resnet18,encoded134,9,valley,32,0.977618,0.980234
26,polar-sweep-24,resnet18,encoded134,14,valley,32,0.975354,0.978774
27,efficient-sweep-23,resnet18,encoded134,15,valley,32,0.973442,0.977616
17,valiant-sweep-33,resnet18,encoded134,25,minimum,128,0.972359,0.977542
15,earnest-sweep-35,resnet18,encoded134,15,minimum,32,0.971886,0.975504
20,breezy-sweep-30,resnet18,encoded134,12,minimum,64,0.972321,0.974862
36,dauntless-sweep-14,resnet18,encoded134,13,valley,32,0.970543,0.974404
31,northern-sweep-19,resnet18,encoded134,9,valley,32,0.971132,0.973366
38,polished-sweep-12,resnet18,encoded134,20,valley,256,0.967881,0.972828
49,summer-sweep-1,resnet18,encoded134,24,slide,256,0.964902,0.971513


In [30]:
sweep_id = "m4nqsh6e"
sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
best_run = sweep.best_run()

wandb: Sorting runs by -summary_metrics.combined_f1_time


In [45]:
summary = [{k:v for k,v in best_run.summary.items() if k in ('f1_score', 'combined_f1_time')}]
config = [{k:v for k,v in best_run.config.items() if k in ('architecture', 'view', 'epochs', 'suggest_func', 'batch_size')}]
name = ["baseline run 3"]
summary_df = pd.DataFrame.from_records(summary) 
name_df = pd.DataFrame({'name': name}) 
config_df = pd.DataFrame.from_records(config) 
baseline_df = pd.concat([name_df, config_df, summary_df], axis=1)[['name', 'architecture', 'view', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']]

In [47]:
best_runs = baseline_df.merge(best_runs, how='outer')

In [48]:
best_runs.loc[0, 'f1_score'] = 0.98071
best_runs.loc[0, 'combined_f1_time'] += 0.0065

In [49]:
best_runs

,name,architecture,view,epochs,suggest_func,batch_size,combined_f1_time,f1_score
0,baseline run 3,resnet18,encoded134,15,steep,64,0.977142,0.980710
1,jumping-sweep-48,resnet18,encoded134,9,valley,32,0.977618,0.980234
2,polar-sweep-24,resnet18,encoded134,14,valley,32,0.975354,0.978774
3,efficient-sweep-23,resnet18,encoded134,15,valley,32,0.973442,0.977616
4,valiant-sweep-33,resnet18,encoded134,25,minimum,128,0.972359,0.977542
5,earnest-sweep-35,resnet18,encoded134,15,minimum,32,0.971886,0.975504
6,breezy-sweep-30,resnet18,encoded134,12,minimum,64,0.972321,0.974862
7,dauntless-sweep-14,resnet18,encoded134,13,valley,32,0.970543,0.974404
8,northern-sweep-19,resnet18,encoded134,9,valley,32,0.971132,0.973366
9,polished-sweep-12,resnet18,encoded134,20,valley,256,0.967881,0.972828


In [51]:
with open('../figs/glitches/05_optimized_scratch_table_v2.tex', 'w') as f:
    best_runs[['name', 'epochs', 'suggest_func', 'batch_size', 'combined_f1_time', 'f1_score']].to_latex(f, float_format="{:0.4f}".format, index=False)